# How I am calculating the proportions

Here I will describe how I generated Figure 2E. If interested the exact script I used to generate this figure is [here](https://gist.github.com/jfear/2680a4ea76fe84e3e4db125e8d688bfc).

In [1]:
# Library imports
from IPython.display import display
import numpy as np
import pandas as pd
from scipy.stats import fisher_exact

# Project level imports
from larval_gonad.notebook import Nb

pd.options.display.max_rows = 5

In [2]:
# A bunch of config information and helpers.
nbconfig = Nb.setup_notebook(seurat_dir='../output/scrnaseq-wf/scrnaseq_combine_force')

last updated: 2018-09-05 
Git hash: 779e2fff250777a09bac38c8f9babe3a884addae


## Data description

In [3]:
# List of germline cluster names
germ_cells = nbconfig.cluster_order[:4]
print('I am using the following germ cell clusters: {}'.format(', '.join(germ_cells)))

I am using the following germ cell clusters: Spermatogonia, Early 1º Spermatocytes, Mid 1º Spermatocytes, Late 1º Spermatocytes


There are several sources of data that I used for these calculations. 

1. I used Seurat [FindMarkers](https://github.com/satijalab/seurat/blob/65b77a9480281ef9ab1aa8816f7c781752092c18/R/differential_expression.R#L75-L321) to generate the differential expression (DEG) table. Seurat uses individual cell-level information for this analysis. The DEG table includes p-values and log fold-change for each gene that was analyzed (Note there may be different numbers of genes for each comparison). I ran the following comparisons:
    * Gonia vs All Cytes: to get the gonia biased genes.
    * Gonia vs Early: to get the Early biased genes.
    * Early vs Mid: to get the Mid biased genes.
    * Mid vs Late: to get the Late biased genes.
    
2. Because individual cell-level counts are low, I have also been using cluster (cell type) level counts. To generate these I summed reads for all cells in each cluster. I use TPM normalization to account for differences in number of cells for each cluster.


The TPM data looks like this:

In [4]:
# Import TPM normalized read counts and take the log.
tpm = np.log10(pd.read_parquet("../output/scrnaseq-wf/tpm.parquet", columns=germ_cells) + 1)
tpm

,Spermatogonia,Early 1º Spermatocytes,Mid 1º Spermatocytes,Late 1º Spermatocytes
FBgn,,,,
FBgn0031081,0.004615,0.015974,0.006159,0.012452
FBgn0031085,0.991906,2.020985,2.212175,2.105093
...,...,...,...,...
FBgn0040723,0.000000,0.017233,0.000000,0.113458
FBgn0031277,0.000000,0.000000,0.028259,0.055727


### Expressed Genes

For Figure 2 I have been using a background set defined as any gene with ≥ 1 read in any germ line cluster.

In [5]:
# Create a list of FBgns that have at least one read in a germline cluster.
expressed = tpm.index[tpm.iloc[:, :4].sum(axis=1) > 0].tolist()
print('There are {:,} genes that have at least 1 read in a germline cluster.'.format(len(expressed)))

There are 14,371 genes that have at least 1 read in a germline cluster.


## Creation of table in Slide 8

Now that I have the general datasets defined I begin to process them. First I import each of the DEG comparisons and generate the counts tables that I showed in slide 8 and sent to you as ('2018-03-31_lineage_comparison.tsv'). I consider a gene DEG if the `p_val_adj <= 0.01`, and I use `avg_logFC` to assign direction.

In [6]:
def diffs(up, down):
    # pull in significant DEG for comparison of up vs down
    dat = pd.read_csv(f'../output/scrnaseq-wf/{up}_vs_{down}.tsv', sep='\t', index_col=0).query('p_val_adj <= 0.01')
    
    # Merge on chromosome information
    dat = dat.join(nbconfig.fbgn2chrom)
    
    # Assign direction using logFC, this creates a bool array where True = 1 and False = 0.
    dat[f'{up}'] = dat.avg_logFC > 0
    dat[f'{down}'] = dat.avg_logFC < 0

    # Set the chromosome order that I want to output
    chrs = ['chrX', 'chr2L', 'chr2R', 'chr3L', 'chr3R', 'chr4', 'chrY', 'chrM']
    
    # Group by chromosome and count the number of DEG genes by summing a bool array.
    df = dat[[f'{up}', f'{down}', 'chrom']].groupby('chrom').sum().reindex(chrs)
    df.columns = pd.MultiIndex.from_arrays([(f'{up}_vs_{down}', f'{up}_vs_{down}'), (f'{up}', f'{down}')])
    return df.fillna(0)

lincomp = pd.concat([diffs('gonia', 'early'), diffs('early', 'mid'), diffs('mid', 'late')], axis=1)
with pd.option_context('display.max_rows', 10):
    display(lincomp)

gonia_vs_early        early_vs_mid        mid_vs_late     
               gonia  early        early    mid         mid late
chrom                                                           
chrX           171.0  266.0        111.0   10.0         0.0  1.0
chr2L          207.0  468.0        158.0  103.0         2.0  6.0
chr2R          265.0  433.0        142.0  106.0         5.0  5.0
chr3L          206.0  416.0        126.0   75.0         6.0  3.0
chr3R          257.0  506.0        174.0  104.0         4.0  7.0
chr4            14.0    5.0          7.0    0.0         0.0  0.0
chrY             0.0    5.0          0.0    0.0         0.0  0.0
chrM             0.0    6.0          0.0    0.0         0.0  7.0

## Creation of Flag Table

After our discussion in that meeting and futher discussions with Brian, we decided to change things up a bit for Figure 2E. He was concerned that Gonia vs Early was not the best way to get the Gonia biased (upregulated) genes, and instead wanted to use Gonia vs All 1º Spermatocytes. As you point out, to run the stats we need out background that I describe above as genes with ≥ 1 read in any germline. **NOTE: The table in the table I gave you (`2018-08-31_flag_lineage_comparison.tsv`) I forgot to add on the background genes so it is short.**

In [7]:
def diffs2(up, down):
    # read in sig DEG.
    dat = pd.read_csv(f'../output/scrnaseq-wf/{up}_vs_{down}.tsv', sep='\t', index_col=0).query('p_val_adj <= 0.01')
    
    if down == 'cytes':
        # Give the gonia biased genes (which are logFC > 0)
        gonia = dat.avg_logFC > 0
        gonia.name = 'gonia_bias'
        return gonia
    else:
        # otherwise return the genes biased towards the later stage (which are logFC < 0)
        cyte = dat.avg_logFC < 0
        cyte.name = f'{down}_bias'
        return cyte

# make an aggregated table for each comparison
dfs = []
for up, down in [('gonia', 'cytes'), ('gonia', 'early'), ('early', 'mid'), ('mid', 'late')]:
    dfs.append(diffs2(up, down))

df = pd.concat(dfs, axis=1, sort=True)
df.index.name = 'FBgn'

# Now add on the expressed genes as background and add on chromosome info
df = df.reindex(expressed).fillna(False).join(nbconfig.fbgn2chrom, how='inner')

# Drop chrM and chrY
df = df.query('chrom != "chrM" & chrom != "chrY"')
df.to_csv('../output/2018-09-05_flag_lineage_comparison.tsv', sep='\t') # save new version of the table.
df

,gonia_bias,early_bias,mid_bias,late_bias,chrom
FBgn,,,,,
FBgn0031081,False,False,False,False,chrX
FBgn0031085,False,True,False,False,chrX
...,...,...,...,...,...
FBgn0040723,False,False,False,False,chr2L
FBgn0031277,False,False,False,False,chr2L


## Testing for differences of DEG by chromosomes

As an example of how I did the Fisher's exact test I will focus on the `early_bias`.

In [8]:
# Pull out early biased column from `df` above.
early = df[['early_bias', 'chrom']]

# Count the number of genes that were DEG by summing bool array.
num_deg = early.groupby('chrom').sum().early_bias
num_deg.name = 'Early DEG'

# Count the total number of genes and subtract `num_deg` to get the `num_not_deg`.
num_not_deg = early.groupby('chrom').size() - num_deg
num_not_deg.name = 'Not Sig'

# combine
deg_table = pd.concat([num_deg, num_not_deg], axis=1, sort=True)

# Verify my math that to total number of genes is equal to the number of genes in early
assert deg_table.sum().sum() == early.shape[0]

with pd.option_context('display.max_rows', 10):
    display(deg_table)

,Early DEG,Not Sig
chrom,,
chr2L,468.0,2316.0
chr2R,433.0,2440.0
chr3L,416.0,2466.0
chr3R,506.0,2982.0
chr4,5.0,96.0
chrX,266.0,1901.0


### Aggregation of Autosomes

For Figure 2E I aggregated the autsomes and tested the X and 4 separately. 

In [9]:
# Aggreage and separate
autosomes = deg_table.loc[['chr2L', 'chr2R', 'chr3L', 'chr3R']].sum()
autosomes.name = 'Autosomes'

x = deg_table.loc['chrX']
x.name = 'X'

_4 = deg_table.loc['chr4']
_4.name = '4'

# X cross tab table
ct = pd.concat([x, autosomes], axis=1)
display(ct)

# 4th cross tab table
ct2 = pd.concat([_4, autosomes], axis=1)
display(ct2)

,X,Autosomes
Early DEG,266.0,1823.0
Not Sig,1901.0,10204.0


,4,Autosomes
Early DEG,5.0,1823.0
Not Sig,96.0,10204.0


#### One-Sided Fisher's Exact Test

I used the one-sided [Fisher's Exact Test'](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.fisher_exact.html) and by passing the `'less'` keyword.

In [10]:
# Run test of depletion on the X
t, pval = fisher_exact(ct.values, 'less')
print(f'The X chromosome shows significant depletion for genes biased towards the early 1º spermatocytes compared to gonia (p-value = {pval:0.6f})')

The X chromosome shows significant depletion for genes biased towards the early 1º spermatocytes compared to gonia (p-value = 0.000218)


In [11]:
# Run test of depletion on the 4th

t, pval = fisher_exact(ct2.values, 'less')
print(f'The 4th chromosome shows significant depletion for genes biased towards the early 1º spermatocytes compared to gonia (p-value = {pval:0.6f})')

The 4th chromosome shows significant depletion for genes biased towards the early 1º spermatocytes compared to gonia (p-value = 0.001259)


#### Two-Sided Fisher's Exact Test

However I could have also used the two sided Fisher's exact test by passing the `'two-sided'` keyword. However, I always find the two-sided test a little harder to explain because it is just showing a difference among cell counts.

In [12]:
# Run test of depletion on the X
t, pval = fisher_exact(ct.values, 'two-sided')
print(f'Two-Sided test for difference of X vs Autosomes when comparing early 1º spermatocytes to gonia (p-value = {pval:0.6f})')

Two-Sided test for difference of X vs Autosomes when comparing early 1º spermatocytes to gonia (p-value = 0.000421)


In [13]:
# Run test of depletion on the 4
t, pval = fisher_exact(ct2.values, 'two-sided')
print(f'Two-Sided test for difference of 4th vs Autosomes when comparing early 1º spermatocytes to gonia (p-value = {pval:0.6f})')

Two-Sided test for difference of 4th vs Autosomes when comparing early 1º spermatocytes to gonia (p-value = 0.002921)


## Each autosome

An alternative would be to compare the X and 4th to each autosomal arm. When comparing chrX vs chr3L significance not as strong as comparing the aggregate, but it is the same.

In [14]:
def comp_auto(target, auto_arm, alternative='less'):
    _target = deg_table.loc[target]
    auto = deg_table.loc[auto_arm]
    ct = pd.concat([_target, auto], axis=1, sort=True)
    _, pval = fisher_exact(ct.values, alternative)
    
    return target, auto_arm, pval

### One-sided

In [15]:
res = []
for comp in [ ('chrX', 'chr2L'), ('chrX', 'chr2R'), ('chrX', 'chr3L'), ('chrX', 'chr3R')]:
    res.append(comp_auto(*comp))
    
pd.DataFrame(res, columns=['target', 'autosome', 'p-val'])

,target,autosome,p-val
0,chrX,chr2L,0.000004
1,chrX,chr2R,0.002448
2,chrX,chr3L,0.014389
3,chrX,chr3R,0.009441


In [16]:
res = []
for comp in [ ('chr4', 'chr2L'), ('chr4', 'chr2R'), ('chr4', 'chr3L'), ('chr4', 'chr3R')]:
    res.append(comp_auto(*comp))
    
pd.DataFrame(res, columns=['target', 'autosome', 'p-val'])

,target,autosome,p-val
0,chr4,chr2L,0.000357
1,chr4,chr2R,0.001508
2,chr4,chr3L,0.002502
3,chr4,chr3R,0.002310


### Two-sided

In [17]:
res = []
for comp in [ ('chrX', 'chr2L'), ('chrX', 'chr2R'), ('chrX', 'chr3L'), ('chrX', 'chr3R')]:
    res.append(comp_auto(*comp, alternative='two-sided'))
    
pd.DataFrame(res, columns=['target', 'autosome', 'p-val'])

,target,autosome,p-val
0,chrX,chr2L,0.000008
1,chrX,chr2R,0.004503
2,chrX,chr3L,0.027473
3,chrX,chr3R,0.018719


In [18]:
res = []
for comp in [ ('chr4', 'chr2L'), ('chr4', 'chr2R'), ('chr4', 'chr3L'), ('chr4', 'chr3R')]:
    res.append(comp_auto(*comp, alternative='two-sided'))
    
pd.DataFrame(res, columns=['target', 'autosome', 'p-val'])

,target,autosome,p-val
0,chr4,chr2L,0.000561
1,chr4,chr2R,0.002512
2,chr4,chr3L,0.005163
3,chr4,chr3R,0.003717


## Output TPM table and expressed gene lists for Maria.

I am going to save out the full tpm table and my expressed genes to send to you.

In [19]:
pd.read_parquet('../output/scrnaseq-wf/tpm.parquet').to_csv('../output/2018-09-05_tpm.tsv', sep='\t')

In [20]:
with open('../output/2018-09-05_expressed_genes.txt', 'w') as fh:
    fh.write('\n'.join(expressed))